# Evaluate building completeness

In support of an ongoing project in Nigeria, we are investigating and evaluating the results of a building digitization effort. The following analyses will be performed to assess the completeness of the digitization effort.

1. Attribute evaluation  
   a. ensure variables in verification protocol are present  
   b. ensure relationship between parcel owner and building occupants  
2. Coverage evaluation  
   a. Calculate across the area in a consistently sized grid (250m)  
   b. Compare building density to Google Buildings  

In [1]:
import sys, os, importlib
import folium

import pandas as pd
import geopandas as gpd
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.misc as misc

from shapely.geometry import Point, box
from shapely.wkt import loads

from math import ceil
import numpy as np
from shapely.geometry import Polygon

import bldg_helper as helper

%load_ext autoreload
%autoreload 2

In [2]:
in_folder =  "/home/wb411133/projects/NGA_buildings/Edo"
out_folder = os.path.join(in_folder, "Results")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

#Define inputs
in_da = os.path.join(os.path.dirname(in_folder), "NGA_buildings_merge.shp")
in_aoi = os.path.join(os.path.dirname(in_folder), 'NGA_buildings_aois.shp')
in_buildings = os.path.join(in_folder, "Collected","SFTAS_ DLI 11.3_ Edo State", "Ca51_Buildings_MEGA.shp")
    
# Define paramters
crs = 3857
m_crs = f'epsg:{crs}' # projection used to calculate metre measurements
res = 250 # resolution of 

# Define outputs
summary_grid    = os.path.join(out_folder, f"Edo_summary_grid_{res}.shp")
out_google_csv  = os.path.join(out_folder, "google_buildings.csv")
out_google_bldg = os.path.join(out_folder, "google_buildings.geojson")
da_buildings    = os.path.join(out_folder, "da_buildings.shp")

In [3]:
inAOI = gpd.read_file(in_aoi)
inAOI = inAOI.loc[inAOI['Name'].apply(lambda x: "Edo" in x)]
inAOI

,id,Name,geometry
6,NaN,Edo,"POLYGON ((5.62855 6.55206, 5.66879 6.55479, 5...."


In [6]:
inB = gpd.read_file(in_buildings)

if inB.crs.to_epsg() != crs:
    inB = inB.to_crs(m_crs)

print(inB.shape)    
inB = inB.loc[inB['geometry'].apply(lambda x: not x is None)]
print(inB.shape)

(95617, 31)
(95617, 31)


In [7]:
inB.head()

,objectid_1,objectid,bld_name,bld_tag,lga,street_nam,off_street,bld_type,bld_purpos,bld_comple,...,land_size,parcel_id,bld_comp_1,for_import,imported,ward_code,file_numbe,Lat,Long,geometry
0,102976,36431,None,EDO/OVIA NORTH EAST/NILL/BLD-MTW874,OVIA NORTH EAST,None,MONDAY OMOROGIE STREET,Block of Flats,RESIDENTIAL,Fully Developed,...,452.22,120665.0,1.0,Yes,Yes,EDSAKA08,EDL 281351,6.431802,5.588422,POINT (622100.295 717493.374)
1,97694,1004,None,EDO/EGOR/EGHOBOR/BLD-UUF009,EGOR,EGHOBOR STREET,CHRISTMARY STREET,Bungalow,RESIDENTIAL,Fully Developed,...,758.56,115278.0,1.0,Yes,Yes,EDSUSL07,EDL 276360,6.359988,5.613039,POINT (624840.646 709449.032)
2,100033,3372,None,EDO/EGOR/QUARLITY/BLD-LRY976,EGOR,QUARLITY,BTC ROAD,Bungalow,RESIDENTIAL,Fully Developed,...,651.52,114670.0,1.0,Yes,Yes,EDSUSL10,EDL 276362,6.380974,5.631827,POINT (626932.096 711799.671)
3,99248,2584,None,"EDO/EGOR/1ST LANE RIGHT, OFF MECHANIC RD,UPPER...",EGOR,MECHANIC ROAD,UPPER SILUKO,Bungalow,RESIDENTIAL,Fully Developed,...,1421.37,110408.0,1.0,Yes,Yes,EDSUSL09,EDL 276363,6.370838,5.566272,POINT (619634.530 710664.331)
4,97935,1246,None,EDO/EGOR/AGASOMWAN/BLD-RTV439,EGOR,AGHASOMWAN STREET,EVBAREKE SPARE PARTS ROAD,Bungalow,RESIDENTIAL,Fully Developed,...,529.12,109187.0,1.0,Yes,Yes,EDSUSL07,EDL 276366,6.365909,5.610673,POINT (624577.231 710112.258)


In [8]:
inB.columns

Index(['objectid_1', 'objectid', 'bld_name', 'bld_tag', 'lga', 'street_nam',
       'off_street', 'bld_type', 'bld_purpos', 'bld_comple', 'state_name',
       'govt_priva', 'bld_num', 'bld_area', 'no_floor_1', 'gdb_geomat', 'area',
       'billing_ar', 'building_v', 'land_value', 'bldg_tag2', 'land_size',
       'parcel_id', 'bld_comp_1', 'for_import', 'imported', 'ward_code',
       'file_numbe', 'Lat', 'Long', 'geometry'],
      dtype='object')

# Calculate general statistics

In [9]:
print(f'Total number of buildings: {inB.shape[0]}')
print(f'Total area of buildings: {inB["bld_area"].sum()}')

Total number of buildings: 95617
Total area of buildings: 23698150.316837884


# Extract buildings from Digitize Africa


In [10]:
importlib.reload(helper)
if not os.path.exists(da_buildings):
    inDA = helper.extract_da_buildings(inAOI, in_da, m_crs)
    inDA.to_file(da_buildings)
else:
    inDA = gpd.read_file(da_buildings)

# Assess attribute completeness
1. Size of the parcel (based on polygon)
2. **Number of structures in parcel, number of occupancy units**  
   a. This is not found in the parcel datasets, but can be calculated.
3. The use of each structure and unit
4. The name, ID, and contact details for the owner of the parcel, and the owner or occupier of the structure, or unit


In [11]:
# Calculate null values in buildings dataset
print(f'Total Records: {inB.shape[0]}')
for col in inB.columns:
    cur_col = inB.loc[:,col]
    print(f'{col}: {cur_col.isna().sum()}')

Total Records: 95617
objectid_1: 0
objectid: 0
bld_name: 76367
bld_tag: 0
lga: 0
street_nam: 1034
off_street: 8765
bld_type: 1
bld_purpos: 0
bld_comple: 1
state_name: 0
govt_priva: 0
bld_num: 0
bld_area: 0
no_floor_1: 0
gdb_geomat: 95617
area: 0
billing_ar: 0
building_v: 280
land_value: 216
bldg_tag2: 0
land_size: 78
parcel_id: 11581
bld_comp_1: 1
for_import: 0
imported: 4625
ward_code: 35
file_numbe: 7892
Lat: 0
Long: 0
geometry: 0


# Summarize buildings within a grid

Create a 250m grid across the study arera and summarize bulidngs and parcels within

In [12]:
# Generate the grid
if not os.path.exists(summary_grid):
    grid = helper.generate_grid(inAOI, res, m_crs)
    grid.to_file(summary_grid)
else:
    grid = gpd.read_file(summary_grid)

In [15]:
if inB.crs.to_epsg() != grid.crs.to_epsg():
    inB = inB.to_crs(grid.crs)
    
grid = helper.summarize_in_grid(grid, inDA, inB, inP=None)

In [16]:
grid['per_b'].max()

107.0

In [17]:
grid.head()

,index,rowIdx,colIdx,geometry,per_b,per_da
0,0,611011.796277,694835.149605,"POLYGON ((611011.796 694835.150, 611261.796 69...",0.0,0.0
1,1,611011.796277,695085.149605,"POLYGON ((611011.796 695085.150, 611261.796 69...",0.0,3.0
2,2,611011.796277,695335.149605,"POLYGON ((611011.796 695335.150, 611261.796 69...",0.0,5.0
3,3,611011.796277,695585.149605,"POLYGON ((611011.796 695585.150, 611261.796 69...",0.0,22.0
4,4,611011.796277,695835.149605,"POLYGON ((611011.796 695835.150, 611261.796 69...",0.0,13.0


In [18]:
grid.to_file(summary_grid)

# RETIRED

In [ ]:
# Summarize buildings in parcels
inP['BLDG_I'] = 0
inP['BLDG_C'] = 0
inP['BLDG_I_G'] = 0
inP['BLDG_C_G'] = 0
for idx, row in inP.iterrows():
    # Summarize collected buildings
    potential_buildings = inB.loc[list(b_idx.intersection(row['geometry'].bounds))]
    m_bld = potential_buildings.loc[~potential_buildings.intersects(row['geometry'])]
    i_bld = potential_buildings.loc[potential_buildings.intersects(row['geometry'])]
    c_bld = potential_buildings.loc[potential_buildings['geometry'].apply(lambda x: row['geometry'].contains(x))]    
    inP.loc[idx, 'BLDG_I'] = i_bld.shape[0]
    inP.loc[idx, 'BLDG_C'] = c_bld.shape[0]
    
    # Summarize buildings in Google
    potential_buildings = inG.loc[list(g_idx.intersection(row['geometry'].bounds))]
    m_bld_g = potential_buildings.loc[~potential_buildings.intersects(row['geometry'])]
    i_bld_g = potential_buildings.loc[potential_buildings.intersects(row['geometry'])]
    c_bld_g = potential_buildings.loc[potential_buildings['geometry'].apply(lambda x: row['geometry'].contains(x))]    
    inP.loc[idx, 'BLDG_I_G'] = i_bld_g.shape[0]
    inP.loc[idx, 'BLDG_C_G'] = c_bld_g.shape[0]
    
    ''' # uncomment this section to stop the loop add a specific index in order to run plotting below
    if idx > 3:
        break
    '''

In [ ]:
m = folium.Map(location=[potential_buildings.unary_union.centroid.y, potential_buildings.unary_union.centroid.x], zoom_start=18)
try:
    folium.GeoJson(row['geometry'], 
             style_function=lambda x: {'fillOpacity':0.5, 'fillColor': 'red', 'color':'#520f0a'}).add_to(m)
except:
    pass
try:
    folium.GeoJson(data=m_bld.to_json(), 
              style_function=lambda x: {'fillOpacity':1,'fillColor': '#ebab15', 'stroke': False}).add_to(m)
except:
    pass
try:
    folium.GeoJson(i_bld.to_json(), 
               style_function=lambda x: {'fillOpacity':1, 'fillColor': '#6878ed', 'stroke': False}).add_to(m)
except:
    pass
try:
    folium.GeoJson(c_bld.to_json(), 
               style_function=lambda x: {'fillOpacity':1, 'fillColor': '#68ed76', 'stroke': False}).add_to(m)
except:
    pass

try:
    folium.GeoJson(data=potential_buildings.to_json(), 
              style_function=lambda x: {'fillOpacity':0,'fillColor': '#ebab15'}).add_to(m)
except:
    pass

m
